# Validate the Generated Features

In [1]:
import warnings

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from feature_engineering import *

warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_parquet("./../../data/model_train/2022_05_16_wind_dataset_lags_central_tendency_dispersion.gzip")

In [3]:
df.head()

,DATETIME,TOTAL_PRECIPITATION_mm,ATM_PRESSURE_mB,ATM_PRESSURE_SEA_LEVEL_mB,MAX_ATM_PRESSURE_PREV_HOUR_mB,MIN_ATM_PRESSURE_PREV_HOUR_mB,GLOBAL_RADIATION_Kjm2,AIR_TEMPERATURE_DRY_BULB_Celsius,DEW_POINT_TEMPERATURE_Celsius,MAX_TEMPERATURE_PREV_HOUR_Celsius,MIN_TEMPERATURE_PREV_HOUR_Celsius,DEW_POINT_MAX_TEMPERATURE_PREV_HOUR_Celsius,DEW_POINT_MIN_TEMPERATURE_PREV_HOUR_Celsius,MAX_RELATIVE_HUMIDITY_PREV_HOUR_percentage,MIN_RELATIVE_HUMIDITY_PREV_HOUR_percentage,RELATIVE_HUMIDITY_percentage,WIND_DIRECTION_degrees,WIND_MAX_GUNS_ms,WIND_SPEED_ms,LATITUDE,LONGITUDE,ALTITUDE,YEAR,MONTH,DAY,HOUR,NAME,CODE,TOTAL_PRECIPITATION_mm_lag_1h,ATM_PRESSURE_mB_lag_1h,ATM_PRESSURE_SEA_LEVEL_mB_lag_1h,MAX_ATM_PRESSURE_PREV_HOUR_mB_lag_1h,MIN_ATM_PRESSURE_PREV_HOUR_mB_lag_1h,GLOBAL_RADIATION_Kjm2_lag_1h,AIR_TEMPERATURE_DRY_BULB_Celsius_lag_1h,DEW_POINT_TEMPERATURE_Celsius_lag_1h,MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_1h,MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_1h,DEW_POINT_MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_1h,DEW_POINT_MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_1h,MAX_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_1h,MIN_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_1h,RELATIVE_HUMIDITY_percentage_lag_1h,WIND_DIRECTION_degrees_lag_1h,WIND_MAX_GUNS_ms_lag_1h,WIND_SPEED_ms_lag_1h,TOTAL_PRECIPITATION_mm_lag_2h,ATM_PRESSURE_mB_lag_2h,ATM_PRESSURE_SEA_LEVEL_mB_lag_2h,MAX_ATM_PRESSURE_PREV_HOUR_mB_lag_2h,MIN_ATM_PRESSURE_PREV_HOUR_mB_lag_2h,GLOBAL_RADIATION_Kjm2_lag_2h,AIR_TEMPERATURE_DRY_BULB_Celsius_lag_2h,DEW_POINT_TEMPERATURE_Celsius_lag_2h,MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_2h,MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_2h,DEW_POINT_MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_2h,DEW_POINT_MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_2h,MAX_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_2h,MIN_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_2h,RELATIVE_HUMIDITY_percentage_lag_2h,WIND_DIRECTION_degrees_lag_2h,WIND_MAX_GUNS_ms_lag_2h,WIND_SPEED_ms_lag_2h,TOTAL_PRECIPITATION_mm_lag_3h,ATM_PRESSURE_mB_lag_3h,ATM_PRESSURE_SEA_LEVEL_mB_lag_3h,MAX_ATM_PRESSURE_PREV_HOUR_mB_lag_3h,MIN_ATM_PRESSURE_PREV_HOUR_mB_lag_3h,GLOBAL_RADIATION_Kjm2_lag_3h,AIR_TEMPERATURE_DRY_BULB_Celsius_lag_3h,DEW_POINT_TEMPERATURE_Celsius_lag_3h,MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_3h,MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_3h,DEW_POINT_MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_3h,DEW_POINT_MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_3h,MAX_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_3h,MIN_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_3h,RELATIVE_HUMIDITY_percentage_lag_3h,WIND_DIRECTION_degrees_lag_3h,WIND_MAX_GUNS_ms_lag_3h,WIND_SPEED_ms_lag_3h,TOTAL_PRECIPITATION_mm_lag_4h,ATM_PRESSURE_mB_lag_4h,ATM_PRESSURE_SEA_LEVEL_mB_lag_4h,MAX_ATM_PRESSURE_PREV_HOUR_mB_lag_4h,MIN_ATM_PRESSURE_PREV_HOUR_mB_lag_4h,GLOBAL_RADIATION_Kjm2_lag_4h,AIR_TEMPERATURE_DRY_BULB_Celsius_lag_4h,DEW_POINT_TEMPERATURE_Celsius_lag_4h,MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_4h,MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_4h,DEW_POINT_MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_4h,DEW_POINT_MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_4h,MAX_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_4h,MIN_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_4h,RELATIVE_HUMIDITY_percentage_lag_4h,WIND_DIRECTION_degrees_lag_4h,WIND_MAX_GUNS_ms_lag_4h,WIND_SPEED_ms_lag_4h,TOTAL_PRECIPITATION_mm_lag_5h,ATM_PRESSURE_mB_lag_5h,ATM_PRESSURE_SEA_LEVEL_mB_lag_5h,MAX_ATM_PRESSURE_PREV_HOUR_mB_lag_5h,MIN_ATM_PRESSURE_PREV_HOUR_mB_lag_5h,GLOBAL_RADIATION_Kjm2_lag_5h,AIR_TEMPERATURE_DRY_BULB_Celsius_lag_5h,DEW_POINT_TEMPERATURE_Celsius_lag_5h,MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_5h,MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_5h,DEW_POINT_MAX_TEMPERATURE_PREV_HOUR_Celsius_lag_5h,DEW_POINT_MIN_TEMPERATURE_PREV_HOUR_Celsius_lag_5h,MAX_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_5h,MIN_RELATIVE_HUMIDITY_PREV_HOUR_percentage_lag_5h,RELATIVE_HUMIDITY_percentage_lag_5h,WIND_DIRECTION_degrees_lag_5h,WIND_MAX_GUNS_ms_lag_5h,WIND_SPEED_ms_lag_5h,TOTAL_PRECIPITATION_mm_lag_6h,ATM_PRESSURE_mB_lag_6h,ATM_PRESSURE_SEA_LEVEL_mB_lag_6h,MAX_AT

In [4]:
dataset_size = df.shape[0]
feature_size = df.shape[1]

In [5]:
nulls = df.isnull().sum() / dataset_size
nulls_20_percent = nulls[nulls > 0.20]
cols_nulls_20_percent = nulls_20_percent.index.tolist()
nulls_20_percent

ATM_PRESSURE_SEA_LEVEL_mB_min_4                          0.204327
ATM_PRESSURE_SEA_LEVEL_mB_max_4                          0.204327
ATM_PRESSURE_SEA_LEVEL_mB_min_12                         0.448355
DEW_POINT_TEMPERATURE_Celsius_min_12                     0.218563
DEW_POINT_MAX_TEMPERATURE_PREV_HOUR_Celsius_min_12       0.220331
DEW_POINT_MIN_TEMPERATURE_PREV_HOUR_Celsius_min_12       0.220331
MAX_RELATIVE_HUMIDITY_PREV_HOUR_percentage_min_12        0.219946
MIN_RELATIVE_HUMIDITY_PREV_HOUR_percentage_min_12        0.220008
RELATIVE_HUMIDITY_percentage_min_12                      0.218462
ATM_PRESSURE_SEA_LEVEL_mB_max_12                         0.448355
DEW_POINT_TEMPERATURE_Celsius_max_12                     0.218563
DEW_POINT_MAX_TEMPERATURE_PREV_HOUR_Celsius_max_12       0.220331
DEW_POINT_MIN_TEMPERATURE_PREV_HOUR_Celsius_max_12       0.220331
MAX_RELATIVE_HUMIDITY_PREV_HOUR_percentage_max_12        0.219946
MIN_RELATIVE_HUMIDITY_PREV_HOUR_percentage_max_12        0.220008
RELATIVE_H

In [7]:
df.shape

(256745, 454)

In [8]:
non_null_tresh = np.round(df.drop(columns=cols_nulls_20_percent).shape[1] * 0.9)
non_null_tresh

341.0

In [10]:
df_selected = df.drop(columns=cols_nulls_20_percent).dropna(axis=0,thresh=non_null_tresh)
df_selected.shape

(200245, 379)

## Trying NGBoost with no normalization for Wind Velocity Prediction 1h ahead

train: 2018

validation: 2019

test: 2020

In [30]:
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from ngboost import NGBRegressor
from ngboost.distns import Normal

In [31]:
def feature_target_split(df,features_to_drop,targets_to_drop,target):
    X = df.drop(columns=(features_to_drop+targets_to_drop+target))
    y = df[target]
    return X,y


In [29]:

df_train = df.query("DATETIME.dt.year == 2018")
df_validation = df.query("DATETIME.dt.year == 2019")
df_test = df.query("DATETIME.dt.year == 2020")

# TODO: Remove last values of target from train and validation to avoid small data leakage.

In [36]:
features_to_drop = ['NAME', 'CODE', 'DATETIME', 'YEAR',
                    'HOUR_target_3h', 'HOUR_target_6h']

targets_to_drop = ['WIND_SPEED_ms_target_3h','WIND_SPEED_ms_target_6h', 
                'WIND_DIRECTION_degrees_target_1h', 'WIND_DIRECTION_degrees_target_3h', 
                'WIND_DIRECTION_degrees_target_6h', 'WIND_MAX_GUNS_ms_target_1h',
                'WIND_MAX_GUNS_ms_target_3h', 'WIND_MAX_GUNS_ms_target_6h']

target = ['WIND_SPEED_ms_target_1h']

X_train,y_train = feature_target_split(df_train.dropna(subset=target).fillna(0),features_to_drop,targets_to_drop,target)
X_validation,y_validation = feature_target_split(df_validation.dropna(subset=target).fillna(0),features_to_drop,targets_to_drop,target)

In [37]:
ngb = NGBRegressor(Dist=Normal).fit(X_train, y_train)


[iter 0] loss=1.8786 val_loss=0.0000 scale=1.0000 norm=1.4444


KeyboardInterrupt: 

In [ ]:
Y_preds = ngb.predict(X_validation)
Y_dists = ngb.pred_dist(X_validation)

test_MSE = mean_squared_error(Y_preds, y_validation)
print("Validation MSE", test_MSE)

test_NLL = -Y_dists.logpdf(y_validation.flatten()).mean()
print("Validation NLL", test_NLL)